# Visualization

In [14]:
import collections
import json
import os
import pickle
import string
import warnings
from multiprocessing.pool import ThreadPool
from operator import itemgetter

import igviz as ig
import networkx as nx
import nltk
import pandas as pd
import plotapi
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
import pyecharts.options as opts
import regex
import requests
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from plotapi import SplitChord, Terminus
from plotly.subplots import make_subplots
from pyecharts.charts import WordCloud
from tqdm import tqdm

warnings.filterwarnings('ignore')
nltk.download('stopwords')
plotapi.api_key("d494c31b-ce51-4470-aa8c-7749ac52ac0b")

Your PlotAPI API key has been saved in your local environment. You will not need to set it again.


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/crinstaniev/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [15]:
pio.templates.default = 'ggplot2'


In [16]:
# switch working directory to root
while True:
    if 'ic-gov-visualization' != os.getcwd().split('/')[-1]:
        os.chdir('..')
    else:
        print(f'working dir: {os.getcwd()}')
        break


working dir: /Users/crinstaniev/Dev/ic-gov-visualization


In [17]:
if not os.path.exists('data'):
    os.mkdir('data')

if not os.path.exists('figures'):
    os.mkdir('figures')

## Fetch Proposals

In [18]:
base_url = 'https://ic-api.internetcomputer.org'

In [19]:
# get total proposal counts
res = requests.get(f'{base_url}/api/nns/proposals-count')
proposals_count = json.loads(res.text)['proposals_count']

print(f'Total Proposals: {proposals_count}')


Total Proposals: 101267


In [20]:
TIMEOUT = 10

# fetch proposals
def get_proposal(id):
    url = base_url + '/api/v3/proposals/' + str(id)
    res = requests.get(url, timeout=TIMEOUT)
    res_dict = json.loads(res.text)
    return res_dict

def get_neuron(id):
    url = f'https://ic-api.internetcomputer.org/api/v3/neurons/{id}'
    res = requests.get(url)
    res_dict = json.loads(res.text)
    return res_dict


In [21]:
# check proposals fetched last time
last_id = None
try:
    proposal_last_time = json.load(open('data/proposals.json', 'r'))
    # convert to pandas dataframe and get the largest id
    df = pd.DataFrame(proposal_last_time)
    last_id = df['id'].max()
    print(f"last time proposal id fetched: {last_id}")
except Exception as e:
    print('proposals file not found')


proposals file not found


In [22]:
proposals = []
last_id = 0 if last_id is None else last_id
proposal_ids = [i for i in range(last_id + 1, proposals_count)]

print(f'proposals from {min(proposal_ids)} to {max(proposal_ids)} will be fetched this time. {max(proposal_ids) - min(proposal_ids) + 1} proposals in total.')

failed_proposals = []
success_proposals = []

def print_id(id):
    try:
        res = get_proposal(id)
        proposals.append(dict(
            id=id,
            data=res
        ))
        success_proposals.append(id)
        # print(f'fetched: proposal {id}')
    except Exception as e:
        failed_proposals.append(id)


# enable multithreading for faster download
pool = ThreadPool(50)

for _ in tqdm(pool.imap_unordered(print_id, proposal_ids), total=len(proposal_ids)):
    pass

print(f"fetching ends. {len(success_proposals)} proposals fetched successfully, {len(failed_proposals)} proposals failed.")


proposals from 1 to 101266 will be fetched this time. 101266 proposals in total.


100%|██████████| 101266/101266 [1:35:13<00:00, 17.72it/s]  

fetching ends. 93485 proposals fetched successfully, 7781 proposals failed.


In [23]:
print(f"failed proposals: {failed_proposals}")

failed proposals: f[14, 5, 20, 19, 25, 12, 13, 16, 11, 17, 18, 26, 34, 30, 31, 42, 29, 37, 32, 33, 84, 69, 70, 82, 71, 68, 67, 88, 120, 125, 122, 121, 129, 127, 126, 128, 130, 134, 132, 135, 115, 112, 107, 117, 111, 119, 118, 105, 106, 131, 116, 109, 113, 114, 133, 136, 165, 163, 172, 156, 175, 173, 177, 182, 180, 188, 207, 209, 208, 210, 192, 197, 196, 198, 195, 214, 216, 215, 146, 212, 213, 211, 219, 218, 201, 203, 200, 217, 205, 243, 248, 247, 220, 222, 221, 224, 237, 267, 254, 256, 268, 258, 259, 263, 261, 262, 272, 294, 309, 305, 308, 312, 311, 310, 315, 302, 317, 316, 318, 319, 320, 322, 323, 347, 384, 387, 386, 354, 389, 390, 393, 391, 392, 397, 400, 374, 373, 388, 394, 406, 395, 396, 410, 408, 398, 422, 421, 442, 405, 447, 448, 407, 443, 449, 444, 445, 446, 450, 451, 453, 452, 454, 455, 456, 458, 459, 457, 461, 460, 463, 462, 466, 464, 465, 467, 468, 470, 469, 472, 471, 473, 474, 475, 476, 477, 478, 479, 480, 481, 482, 483, 485, 487, 486, 484, 488, 489, 490, 491, 492, 493, 494,

In [24]:
if last_id != 0:
    last_proposals = pickle.load(open('data/nns_data_raw.pkl', 'rb'))
    proposals.extend(last_proposals)
    print('proposals appended')

In [25]:
pickle.dump(proposals, open('data/nns_data_raw.pkl', 'wb'))

In [26]:
data = pickle.load(open('data/nns_data_raw.pkl', 'rb'))

In [27]:
data = pickle.load(open('data/nns_data_raw.pkl', 'rb'))
data.sort(key=itemgetter('id'))
json.dump(data, open('data/proposals.json', 'w'))

In [28]:
# filter empty data
proposals_no_empty = []

for item in data:
    if (item['data'].get('code') != 404):
        proposals_no_empty.append(
            item['data']
        )
json.dump(proposals_no_empty, open('data/proposals_no_empty.json', 'w'))


In [29]:
df = pd.read_json('data/proposals_no_empty.json').drop_duplicates(subset=['id'])
df

,action,action_nns_function,deadline_timestamp_seconds,decided_timestamp_seconds,executed_timestamp_seconds,failed_timestamp_seconds,id,known_neurons_ballots,latest_tally,payload,...,proposer,reject_cost_e8s,reward_status,settled_at,status,summary,title,topic,updated_at,url
0,ExecuteNnsFunction,NnsCanisterUpgrade,NaN,1620340878,1.620341e+09,0.0,1,[],"{'no': 0, 'timestamp_seconds': 1620340878, 'to...","{'arg': [], 'authz_changes': [], 'canister_id'...",...,35.0,100000000,SETTLED,2021-05-06 16:00:00,EXECUTED,Upgrade ledger canister to git commit 8a560f95...,,TOPIC_NETWORK_CANISTER_MANAGEMENT,2021-08-05 15:50:43.155180,https://github.com/dfinity/nns-proposals/blob/...
1,ExecuteNnsFunction,NnsCanisterUpgrade,NaN,1620340965,1.620341e+09,0.0,2,[],"{'no': 0, 'timestamp_seconds': 1620340965, 'to...","{'arg': [], 'authz_changes': [], 'canister_id'...",...,35.0,100000000,SETTLED,2021-05-06 16:00:00,EXECUTED,Upgrade registry canister to git commit 8a560f...,,TOPIC_NETWORK_CANISTER_MANAGEMENT,2021-08-05 15:50:43.157946,https://github.com/dfinity/nns-proposals/blob/...
2,ExecuteNnsFunction,NnsRootUpgrade,NaN,1620341096,1.620341e+09,0.0,4,[],"{'no': 0, 'timestamp_seconds': 1620341096, 'to...",{},...,35.0,100000000,SETTLED,2021-05-06 16:00:00,EXECUTED,Upgrade root canister to git commit 8a560f9510...,,TOPIC_NETWORK_CANISTER_MANAGEMENT,2021-08-05 15:50:43.159834,https://github.com/dfinity/nns-proposals/blob/...
3,ExecuteNnsFunction,NnsCanisterUpgrade,NaN,1620341144,1.620341e+09,0.0,5,[],"{'no': 0, 'timestamp_seconds': 1620341144, 'to...","{'arg': [], 'authz_changes': [], 'canister_id'...",...,35.0,100000000,SETTLED,2021-05-06 16:00:00,EXECUTED,Upgrade cycles-minting canister to git commit ...,,TOPIC_NETWORK_CANISTER_MANAGEMENT,2021-08-05 15:50:43.160766,https://github.com/dfinity/nns-proposals/blob/...
4,ExecuteNnsFunction,NnsCanisterUpgrade,NaN,1620341195,1.620341e+09,0.0,6,[],"{'no': 0, 'timestamp_seconds': 1620341195, 'to...","{'arg': [], 'authz_changes': [], 'canister_id'...",...,35.0,100000000,SETTLED,2021-05-06 16:00:00,EXECUTED,Upgrade lifeline canister to git commit 8a560f...,,TOPIC_NETWORK_CANISTER_MANAGEMENT,2021-08-05 15:50:43.161765,https://github.com/dfinity/nns-proposals/blob/...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80720,ExecuteNnsFunction,IcpXdrConversionRate,1.673310e+09,1673266864,1.673267e+09,0.0,84447,"[{'id': '28', 'name': 'Internet Computer Assoc...","{'no': 0, 'timestamp_seconds': 1673266864, 'to...","{'data_source': {'icp': ['KuCoin'], 'sdr': 're...",...,25.0,1000000000,ACCEPT_VOTES,NaT,EXECUTED,,The ICP/XDR conversion rate is set to 3.2173.,TOPIC_EXCHANGE_RATE,2023-01-09 14:43:23.130829,
80721,ExecuteNnsFunction,IcpXdrConversionRate,1.673311e+09,1673267465,1.673267e+09,0.0,84448,"[{'id': '28', 'name': 'Internet Computer Assoc...","{'no': 0, 'timestamp_seconds': 1673267465, 'to...","{'data_source': {'icp': ['Gate.io', 'KuCoin'],...",...,24.0,1000000000,ACCEPT_VOTES,NaT,EXECUTED,,The ICP/XDR conversion rate is set to 3.2243.,TOPIC_EXCHANGE_RATE,2023-01-09 14:43:23.112136,
80722,ExecuteNnsFunction,IcpXdrConversionRate,1.673311e+09,1673268065,1.673268e+09,0.0,84449,"[{'id': '28', 'name': 'Internet Computer Assoc...","{'no': 0, 'timestamp_seconds': 1673268065, 'to...","{'data_source': {'icp': ['KuCoin', 'Gate.io'],...",...,22.0,1000000000,ACCEPT_VOTES,NaT,EXECUTED,,The ICP/XDR conversion rate is set to 3.2347.,TOPIC_EXCHANGE_RATE,2023-01-09 14:43:23.094485,
80723,ExecuteNnsFunction,IcpXdrConversionRate,1.673312e+09,1673268664,1.673269e+09,0.0,84450,"[{'id': '28', 'name': 'Internet Computer Assoc...","{'no': 0, 'timestamp_seconds': 1673268664, 'to...","{'data_source': {'icp': ['Huobi'], 'sdr': 'ref...",...,23.0,1000000000,ACCEPT_VOTES,NaT,EXECUTED,,The ICP/XDR conversion rate is set to 3.2398.,TOPIC_EXCHANGE_RATE,2023-01-09 14:43:23.076372,


## Wordcloud - Proposal Summary

In [30]:
with open('data/proposal_summary_text.txt', 'w') as f:
    f.write(' '.join(list(df['summary'].values)))


In [31]:
# cleaning data for wordcloud
df_wordcloud = df[['topic', 'summary']]
df_wordcloud = df_wordcloud.groupby('topic').aggregate(' '.join).reset_index()
df_wordcloud.to_csv('data/topic_summary_joined.csv')

# remove non-utf8 characters


def remove_non_utf8(x: str):
    x = x.encode('utf-8', errors='ignore').decode('utf-8')
    x = regex.sub(r'[^\x00-\x7f]', u'', x)
    x = regex.sub(r"http\S+", "", x)
    x = x.translate(str.maketrans('', '', string.punctuation))
    x = word_tokenize(x)
    x = [word for word in x if not word in stopwords.words('english')]

    return (" ").join(x)


df_wordcloud['summary'] = df_wordcloud['summary'].apply(remove_non_utf8)
df_wordcloud.to_csv('data/topic_summary_joined.csv')


In [32]:
df_wordcloud.head()


,topic,summary
0,TOPIC_EXCHANGE_RATE,The ICPXDR conversion rate set 404904 The ICPX...
1,TOPIC_GOVERNANCE,SetAuthorizedSubnetworks SetAuthorizedSubnetwo...
2,TOPIC_KYC,The proposal unlocks neurons associated listed...
3,TOPIC_NETWORK_CANISTER_MANAGEMENT,Upgrade ledger canister git commit 8a560f9510b...
4,TOPIC_NETWORK_ECONOMICS,ClearProvisionalWhitelist Initialize rewards t...


In [33]:
def generate_wordcloud(df_wordcloud, topics):

    words = []
    for topic in topics:
        words.append(
            df_wordcloud.query(f"topic == '{topic}'")['summary'].values[0]
        )

    all_words = (' ').join(words)

    with open('data/all_words.txt', 'w') as f:
        f.write(all_words)

    wc = WordCloud(
        background_color='black',
        repeat=True,
        width=1024 * 2,
        height=768 * 2,
        max_words=200,
        colormap='Set2',
        font_path='fonts/impact.ttf'
    )
    wc.generate(all_words)

    return wc


In [34]:
def generate_word_freq(df_wordcloud, topics):
    words = []
    for topic in topics:
        words.append(
            df_wordcloud.query(f"topic == '{topic}'")['summary'].values[0]
        )

    all_words = [ele for ele in words if len(ele) > 20]
    all_words = (' ').join(all_words).upper()
    all_words = regex.sub(r'\b[0-9]+\b\W*', '', all_words)
    all_words = regex.sub(r'(?:^| )\w(?:$| )', ' ', all_words)

    def replace_digit(string):
        string = regex.sub(r'\d', '', string).strip()
        return string

    all_words = all_words.upper()

    tokens = nltk.word_tokenize(all_words)
    text = nltk.Text(tokens)
    freq = nltk.Counter(text).items()
    res = []
    for item in freq:
        res.append(item)

    res.sort(key=lambda x: x[1], reverse=True)
    return res


data = generate_word_freq(
    df_wordcloud=df_wordcloud,
    topics=[
        # 'TOPIC_EXCHANGE_RATE',
        'TOPIC_GOVERNANCE',
        # 'TOPIC_KYC',
        # 'TOPIC_NETWORK_CANISTER_MANAGEMENT',
        # 'TOPIC_NETWORK_ECONOMICS',
        # 'TOPIC_NODE_ADMIN',
        # 'TOPIC_NODE_PROVIDER_REWARDS',
        # 'TOPIC_PARTICIPANT_MANAGEMENT',
        'TOPIC_SUBNET_MANAGEMENT',
    ]
)

(
    WordCloud(init_opts=opts.InitOpts())
    .add(
        series_name="Keywords",
        data_pair=data,
        word_size_range=[20, 80],
        textstyle_opts=opts.TextStyleOpts(
            font_family="impact",
            font_weight='bold',
            color='red'
        ),
        rotate_step=90
    )
    .set_global_opts(
        # title_opts=opts.TitleOpts("Theme-dark"),
        title_opts=opts.TitleOpts(
            title="Keywords", title_textstyle_opts=opts.TextStyleOpts(font_size=23),
        ),
        tooltip_opts=opts.TooltipOpts(is_show=True),
    )
    .render('figures/wordcloud.html')
)


'/Users/crinstaniev/Dev/ic-gov-visualization/figures/wordcloud.html'

## Area Chart - Topics River

In [35]:
topic_time_df = df[['proposal_id', 'topic',
                    'updated_at']].set_index('proposal_id')

topic_time_df['updated_at'] = topic_time_df['updated_at'].dt.floor('d')

topic_time_stats = topic_time_df.groupby(
    'updated_at').value_counts().reset_index()
topic_time_stats.columns = ['date', 'topic', 'count']
topic_time_stats


,date,topic,count
0,2021-08-05,TOPIC_EXCHANGE_RATE,135
1,2021-08-05,TOPIC_NODE_PROVIDER_REWARDS,48
2,2021-08-05,TOPIC_SUBNET_MANAGEMENT,37
3,2021-08-05,TOPIC_GOVERNANCE,31
4,2021-08-05,TOPIC_NETWORK_CANISTER_MANAGEMENT,30
...,...,...,...
1351,2023-01-06,TOPIC_EXCHANGE_RATE,145
1352,2023-01-07,TOPIC_EXCHANGE_RATE,142
1353,2023-01-08,TOPIC_EXCHANGE_RATE,144
1354,2023-01-09,TOPIC_EXCHANGE_RATE,201


In [36]:
fig = px.area(topic_time_stats, x='date', y='count',
              color='topic')
fig.update_yaxes(type='log', range=[0, 4])

with open('figures/timeriver.html', 'w') as f:
    f.write(fig.to_html())

fig.show()


In [37]:
topic_time_df = df[['proposal_id', 'topic',
                    'updated_at']].set_index('proposal_id')

topic_time_df['updated_at'] = topic_time_df['updated_at'].dt.floor('d')

topic_time_stats = topic_time_df.groupby(
    'updated_at').value_counts().unstack(fill_value=0).reset_index()
topics = list(topic_time_stats.columns[1:])

topic_time_stats['sum'] = 0

for topic in topics:
    topic_time_stats['sum'] += topic_time_stats[topic]


In [38]:
fig = make_subplots(rows=2, cols=1, shared_xaxes=True)

for topic in topics:
    fig.add_trace(go.Scatter(
        x=topic_time_stats['updated_at'],
        y=topic_time_stats[topic],
        mode='lines',
        line=dict(width=0.5),
        stackgroup='one',
        groupnorm='percent',
        name=topic,
    ), row=2, col=1)

fig.add_traces(
    go.Line(x=topic_time_stats['updated_at'], y=topic_time_stats['sum'], name='Sum of all Topics'))

fig.update_layout(
    dict(title='Proposal Topic Changes Over Time', width=1200, height=800))

# fig.update_layout(yaxis_range=(0, 100))
fig.update_xaxes(title='Time')

fig['layout']['yaxis2'].update(title='Percentage', range=[60, 100])
fig['layout']['yaxis1'].update(title='Count')

fig.show()
with open('figures/topic_area_chart.html', 'w') as f:
    f.write(fig.to_html())


/Users/crinstaniev/.local/share/virtualenvs/ic-gov-visualization-TiLus9tN/lib/python3.10/site-packages/plotly/graph_objs/_deprecations.py:378: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.




## Chord

In [39]:
proposer_network = df[['proposal_id', 'proposer',
                       'known_neurons_ballots', 'updated_at']]

# drop proposals with no votes
proposer_network = proposer_network[proposer_network['known_neurons_ballots'].apply(
    len) != 0]
proposer_network['proposer'] = proposer_network['proposer'].astype(int)

# build proposer roster
voters = proposer_network['known_neurons_ballots'].values.flatten()
voters_temp = set()
voters_id = set()
for voter_group in tqdm(voters):
    for voter in voter_group:
        voters_id.add(int(voter.get('id')))
        voter = (voter.get('id'), voter.get('name'))
        voters_temp.add(voter)

voters = voters_temp

proposer_set = set()
for proposal in proposer_network.itertuples():
    proposer_id = proposal.proposer
    proposer_set.add(int(proposer_id))


100%|██████████| 80709/80709 [00:00<00:00, 339732.97it/s]


In [40]:
chord_df = pd.DataFrame(0, columns=list(voters_id), index=list(proposer_set))

for proposal in tqdm(proposer_network.itertuples(), total=len(proposer_network)):
    voters = proposal.known_neurons_ballots
    proposer_id = int(proposal.proposer)
    for voter in voters:
        voter_id = int(voter.get('id'))
        vote = voter.get('vote')
        chord_df.loc[proposer_id, voter_id] += 1


100%|██████████| 80709/80709 [00:22<00:00, 3619.08it/s] 


In [41]:
chord_df = chord_df[chord_df.sum(axis=1).sort_values() > 100]


In [42]:
links = []
nodes = []

for item in chord_df.itertuples():
    proposer = f'Proposer {item.Index}'
    nodes.append(dict(
        name=proposer,
        group='left'
    ))
    for voter in chord_df.columns:
        amount = int(chord_df.loc[item.Index, voter])
        voter = f'Voter {voter}'
        links.append(dict(
                source=voter,
                target=proposer,
                value=amount
        ))

for voter in chord_df.columns:
    voter = f'Voter {voter}'
    nodes.append(dict(
        name=voter,
        group='right'
    ))

links = list(filter(lambda x : x.get('value') > 100, links))


In [43]:
pd.DataFrame(links).sort_values(by='value')

,source,target,value
139,Voter 6362091663310642824,Proposer 47,111
130,Voter 16335946240875077438,Proposer 40,120
129,Voter 11053086394920719168,Proposer 40,121
137,Voter 12890113924500239096,Proposer 40,128
110,Voter 27,Proposer 38,144
...,...,...,...
31,Voter 28,Proposer 23,15172
73,Voter 28,Proposer 25,15242
72,Voter 27,Proposer 25,15242
10,Voter 28,Proposer 22,15245


In [44]:
SplitChord(
    links,
    nodes,
    # directed=True,
    conjunction='to',
    verb='give',
    noun='votes',
).to_html('figures/chord.html')


## Terminus

In [45]:
few_links = list(filter(lambda x: x['value'] > 1000, links))


In [46]:
Terminus(
    few_links,
    show_stats=False
).to_html('figures/terminus.html')

## Network

In [47]:
df_network = df[df['topic'] != 'TOPIC_EXCHANGE_RATE']['summary']

In [48]:
network_all_words = ('. '.join(list(df_network.values)))
network_all_words = regex.sub(r'\b[0-9]+\b\W*', '', network_all_words)
network_all_words = regex.sub(r'(?:^| )\w(?:$| )', ' ', network_all_words)


In [49]:
def remove_url(txt):
    url_pattern = regex.compile(r'https?://\S+|www\.\S+')
    no_url = url_pattern.sub(r'', txt)

    return no_url


def stem_tokens(tokens):
    stemmer = nltk.stem.PorterStemmer()
    tokens = [stemmer.stem(token) for token in tokens]
    return tokens


def lemmatize_tokens(tokens):
    lemmatizer = nltk.stem.WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token) for token in tokens]
    return tokens


In [50]:
text_no_urls = remove_url(network_all_words)
text_no_urls = text_no_urls.encode('utf-8', errors='ignore').decode('utf-8')
words_in_text = text_no_urls.split()
stop_words = set(stopwords.words('english'))
words_nsw = [word for word in words_in_text if not word in stop_words]
words_nsw = [ele for ele in words_nsw if len(ele) < 20]


In [51]:
lemmatizer = WordNetLemmatizer()
words_nsw_nc = [lemmatizer.lemmatize(w) for w in words_nsw]
words_nsw_nc = [
    word for word in words_nsw_nc if word not in string.punctuation]


In [52]:
bigram = list(nltk.bigrams(words_nsw_nc))
bigram_counts = collections.Counter(bigram).most_common(100)

In [53]:
G = nx.Graph()
for edge in bigram_counts:
    link = edge[0]
    count = edge[1]
    
    G.add_edge(link[0], link[1], weight=count)

In [54]:
fig = ig.plot(
    G,
    node_opacity=0.8,
)
fig.update_layout(
    dict(
        width=800,
        height=600,
    )
)

with open('figures/network.html', 'w') as f:
    f.write(fig.to_html())

fig.show()
